In [1]:
! pip install transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 10.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset , Dataset
import pandas as pd

In [3]:
text_classification_data=load_dataset("dair-ai/emotion" , "split" , split="test")
text_classification_data

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [4]:
text_classification_df=text_classification_data.to_pandas()
text_classification_df

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [5]:
map_emotions={
    0 : "sadness",
    1 : "joy",
    2 : "love",
    3 : "anger",
    4 : "fear",
    5 : "surprise"
    }

text_classification_df["target"] = text_classification_df["label"].apply(lambda x: map_emotions[x])
text_classification_df["source"] = text_classification_df["text"].apply(lambda x: "text-classification-emotion: " + x)

text_classification_df = text_classification_df[["source", "target"]]
text_classification_df

,source,target
0,text-classification-emotion: im feeling rather...,sadness
1,text-classification-emotion: im updating my bl...,sadness
2,text-classification-emotion: i never make her ...,sadness
3,text-classification-emotion: i left with my bo...,joy
4,text-classification-emotion: i was feeling a l...,sadness
...,...,...
1995,text-classification-emotion: i just keep feeli...,anger
1996,text-classification-emotion: im feeling a litt...,anger
1997,text-classification-emotion: i feel that i am ...,joy
1998,text-classification-emotion: im feeling more c...,joy


In [9]:
data = load_dataset("mlqa", "mlqa.hi.en", split="validation")

In [10]:
qa_df=data.to_pandas()
qa_df

,context,question,answers,id
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,What is the name of a type of string that cons...,"{'answer_start': [2267], 'text': ['कोलेन्काइमा']}",60ee75c50c8472be7cce1a24ee2cd7409ee4dd52
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,What are made up almost entirely of parenchyma...,"{'answer_start': [158], 'text': ['पत्तियां']}",561971f7978f678c3d1ba2a946036cdc131c4d49
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,What kind of plants possess xylem?,"{'answer_start': [292], 'text': ['ट्रेकियोफाइट...",18a5c05701b7359fcd32a379b2ac4a9a5d7544de
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,What is the leptome?,"{'answer_start': [969], 'text': ['समान कार्य क...",694ce8b90f6854bcd6c3a505767df90f8b659d06
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",Epidermal cells can do what?,"{'answer_start': [433], 'text': ['क्यटिन का सं...",25be1db3766bfbb5f64b3a8f4bf4af57470eb651
...,...,...,...,...
502,स्पार्टाकस नाम वैसे काले सागर क्षेत्र में साक्...,which King?,"{'answer_start': [89], 'text': ['थ्रेसियन राजव...",c555e59e9cf74869aaf94cbeaf932cd1fd89e55e
503,इस समय तक पोम्पे की टुकड़ी स्पेन लौट चुकी थी औ...,What happened to captured soldiers?,"{'answer_start': [1094], 'text': ['क्रूस पर चढ...",1f79f28190b12342fc7a39cea0202d33af7488c9
504,"आर्गन-39, 269 वर्ष की एक आधा जीवन के साथ, आवेद...",What is the half-life of Argon 39?,"{'answer_start': [10], 'text': ['269 वर्ष']}",cc81fb916f547d7d51730e0f726891179b361ebd
505,जॉनस्टन का जन्म व बचपन ओंटारियो में बीता। वहाँ...,What role in government did Stephen Harper have?,"{'answer_start': [669], 'text': ['प्रधानमंत्री']}",244b552d2f8e850eb0b88d4c7440c575639fa911


In [11]:
qa_df["source"] = qa_df[["context", "question"]].apply(lambda x: "context-question-answering: context: " + x["context"] + " question: " + x["question"], axis=1)
qa_df["target"] = qa_df["answers"].apply(lambda x: x["text"][0])

qa_df = qa_df[["source", "target"]]
qa_df

,source,target
0,context-question-answering: context: पैरेनकाइम...,कोलेन्काइमा
1,context-question-answering: context: पैरेनकाइम...,पत्तियां
2,context-question-answering: context: जाइलेम को...,ट्रेकियोफाइटों
3,context-question-answering: context: फ्लोएम उच...,समान कार्य करने वाला एक अधिक सरल ऊतक
4,context-question-answering: context: सभी हवाई ...,क्यटिन का संश्लेषण
...,...,...
502,context-question-answering: context: स्पार्टाक...,थ्रेसियन राजवंश के राजा
503,context-question-answering: context: इस समय तक...,क्रूस पर चढ़ा दिया गया।
504,"context-question-answering: context: आर्गन-39,...",269 वर्ष
505,context-question-answering: context: जॉनस्टन क...,प्रधानमंत्री


In [12]:
data = load_dataset("squad_v2", split="validation")
qgen_df = data.to_pandas()
qgen_df = qgen_df.sample(500)

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [13]:
qgen_df

,id,title,context,question,answers
731,5705f13d52bb8914006896f2,Southern_California,"In 1900, the Los Angeles Times defined souther...",In which year did the newspaper change its pre...,"{'text': ['1999', '1999', '1999'], 'answer_sta..."
10382,57309446396df919000961b9,Imperialism,"The Age of Imperialism, a time period beginnin...",What were European countries doing during the ...,"{'text': ['colonizing', 'process of colonizing..."
9491,5ad2bb5fd7d075001a42a083,Rhine,"In World War II, it was recognised that the Rh...",How long did the Cold War last?,"{'text': [], 'answer_start': []}"
9503,572fac17947a6a140053cb54,Scottish_Parliament,"Following a referendum in 1997, in which the S...",When was the current parliament of Scotland co...,"{'text': ['Following a referendum in 1997', '1..."
1476,5ad263d9d7d075001a429126,Huguenot,"In the early years, many Huguenots also settle...","In what year was Charleston, North Carolina fo...","{'text': [], 'answer_start': []}"
...,...,...,...,...,...
3997,5725d183271a42140099d23f,"Fresno,_California","Before World War II, Fresno had many ethnic ne...","In 1942, what was North Fresno previously called?","{'text': ['Pinedale', 'Pinedale', 'Pinedale'],..."
1120,5a2c0fb8bfd06b001a5ae9bb,Sky_(United_Kingdom),BSkyB's digital service was officially launche...,Within 30 days how many digiboxes had been dis...,"{'text': [], 'answer_start': []}"
6111,5a679db8f038b7001ab0c350,Private_school,"In Ireland, private schools (Irish: scoil phrí...",How are school fees in the rest of the world c...,"{'text': [], 'answer_start': []}"
8883,5a893a003b2508001a72a53f,Prime_number,"In addition to the Riemann hypothesis, many mo...",Which conjecture holds that every odd integer ...,"{'text': [], 'answer_start': []}"


In [14]:
qgen_df["source"] = qgen_df["context"].apply(lambda x: "context-question-generation: " + x)
qgen_df["target"] = qgen_df["question"]

In [15]:
qgen_df = qgen_df[["source", "target"]]
qgen_df

,source,target
731,"context-question-generation: In 1900, the Los ...",In which year did the newspaper change its pre...
10382,context-question-generation: The Age of Imperi...,What were European countries doing during the ...
9491,"context-question-generation: In World War II, ...",How long did the Cold War last?
9503,context-question-generation: Following a refer...,When was the current parliament of Scotland co...
1476,context-question-generation: In the early year...,"In what year was Charleston, North Carolina fo..."
...,...,...
3997,context-question-generation: Before World War ...,"In 1942, what was North Fresno previously called?"
1120,context-question-generation: BSkyB's digital s...,Within 30 days how many digiboxes had been dis...
6111,"context-question-generation: In Ireland, priva...",How are school fees in the rest of the world c...
8883,context-question-generation: In addition to th...,Which conjecture holds that every odd integer ...


In [16]:
data = load_dataset("paws", "labeled_final", split="validation")
trans_df = data.to_pandas()
trans_df = trans_df.sample(1000)
trans_df

README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

,id,sentence1,sentence2,label
3867,3868,"It was granted in 1544 to John Sone , who imme...",It was granted to Henry Audeley and John Corda...,0
2517,2518,The forewings are dirty straw dark with three ...,The forewings are dirty straw dark with three ...,1
3864,3865,`` Dawn '' is the thirty-second episode ( prod...,`` Dawn '' is the thirty-ninth episode ( produ...,0
1541,1542,Some of them are described here and some of th...,Some of them are described below and some of t...,0
1401,1402,Tebenna carduiella is a moth of the Choreutida...,Tebenna carduiella is a moth of the Choreutida...,1
...,...,...,...,...
6178,6179,Sepahua is a district in southern Peru in Atal...,Sepahua is a district in southern Peru in the ...,1
5291,5292,Karen Lieve Ria Hostens is married to J. Peter...,"Ria Hostens is married to J. Peter Burgess , c...",1
1869,1870,Vempati Chinna Satyam ( 15 October 1929 -- 29 ...,"Vempati Chinna Satyam ( October 15 , 1929 -- J...",1
4137,4138,While the exclusion of black players was not a...,While the exclusion of African players was not...,0


In [17]:
trans_df = trans_df[trans_df["label"]==1]
trans_df = trans_df[["sentence1", "sentence2"]]
trans_df

,sentence1,sentence2
2517,The forewings are dirty straw dark with three ...,The forewings are dirty straw dark with three ...
1401,Tebenna carduiella is a moth of the Choreutida...,Tebenna carduiella is a moth of the Choreutida...
6657,Tuljapur is an earth dam on Bori - River near ...,Tuljapur is an earthfill dam on Bori river nea...
6496,The station was founded by the British Falklan...,The station was established by the British Fal...
6880,Jonas Björkman and Fabrice Santoro won against...,Jonas Björkman and Fabrice Santoro won in the ...
...,...,...
258,"From 1863 to 1866 , Dunedin and Suburbs North ...",Dunedin and Suburbs North was a parliamentary ...
2549,"The new year , the spring , the wine and the b...","Also , `` The new year , the spring , the wine..."
6178,Sepahua is a district in southern Peru in Atal...,Sepahua is a district in southern Peru in the ...
5291,Karen Lieve Ria Hostens is married to J. Peter...,"Ria Hostens is married to J. Peter Burgess , c..."


In [18]:
trans_df["source"] = trans_df["sentence1"].apply(lambda x: "paraphrase: " + x)
trans_df["target"] = trans_df["sentence2"]
trans_df

,sentence1,sentence2,source,target
2517,The forewings are dirty straw dark with three ...,The forewings are dirty straw dark with three ...,paraphrase: The forewings are dirty straw dark...,The forewings are dirty straw dark with three ...
1401,Tebenna carduiella is a moth of the Choreutida...,Tebenna carduiella is a moth of the Choreutida...,paraphrase: Tebenna carduiella is a moth of th...,Tebenna carduiella is a moth of the Choreutida...
6657,Tuljapur is an earth dam on Bori - River near ...,Tuljapur is an earthfill dam on Bori river nea...,paraphrase: Tuljapur is an earth dam on Bori -...,Tuljapur is an earthfill dam on Bori river nea...
6496,The station was founded by the British Falklan...,The station was established by the British Fal...,paraphrase: The station was founded by the Bri...,The station was established by the British Fal...
6880,Jonas Björkman and Fabrice Santoro won against...,Jonas Björkman and Fabrice Santoro won in the ...,paraphrase: Jonas Björkman and Fabrice Santoro...,Jonas Björkman and Fabrice Santoro won in the ...
...,...,...,...,...
258,"From 1863 to 1866 , Dunedin and Suburbs North ...",Dunedin and Suburbs North was a parliamentary ...,"paraphrase: From 1863 to 1866 , Dunedin and Su...",Dunedin and Suburbs North was a parliamentary ...
2549,"The new year , the spring , the wine and the b...","Also , `` The new year , the spring , the wine...","paraphrase: The new year , the spring , the wi...","Also , `` The new year , the spring , the wine..."
6178,Sepahua is a district in southern Peru in Atal...,Sepahua is a district in southern Peru in the ...,paraphrase: Sepahua is a district in southern ...,Sepahua is a district in southern Peru in the ...
5291,Karen Lieve Ria Hostens is married to J. Peter...,"Ria Hostens is married to J. Peter Burgess , c...",paraphrase: Karen Lieve Ria Hostens is married...,"Ria Hostens is married to J. Peter Burgess , c..."


In [19]:
trans_df = trans_df[["source", "target"]]
trans_df


,source,target
2517,paraphrase: The forewings are dirty straw dark...,The forewings are dirty straw dark with three ...
1401,paraphrase: Tebenna carduiella is a moth of th...,Tebenna carduiella is a moth of the Choreutida...
6657,paraphrase: Tuljapur is an earth dam on Bori -...,Tuljapur is an earthfill dam on Bori river nea...
6496,paraphrase: The station was founded by the Bri...,The station was established by the British Fal...
6880,paraphrase: Jonas Björkman and Fabrice Santoro...,Jonas Björkman and Fabrice Santoro won in the ...
...,...,...
258,"paraphrase: From 1863 to 1866 , Dunedin and Su...",Dunedin and Suburbs North was a parliamentary ...
2549,"paraphrase: The new year , the spring , the wi...","Also , `` The new year , the spring , the wine..."
6178,paraphrase: Sepahua is a district in southern ...,Sepahua is a district in southern Peru in the ...
5291,paraphrase: Karen Lieve Ria Hostens is married...,"Ria Hostens is married to J. Peter Burgess , c..."


In [20]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 25.8 MB/s eta 0:00:00


In [21]:
data = load_dataset("samsum", split="test")
summ_df = data.to_pandas()
summ_df = summ_df[:500]
summ_df

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....
...,...,...,...
495,13828505,"Amber: Hi Erin, guess what, John and Annie are...",John and Annie are moving to London because he...
496,13716957,Vicky: Is everyone still at town sq? I might h...,"They are meeting at Fratellis, upstairs."
497,13829569,Kaylin: <file_gif> \r\nKaylin: that's what app...,Kaylin has set her alarm for tomorrow.
498,13729275,Keira: How come it started with wildangel3 the...,Keira wonders why wildangel plays in the wrong...


In [22]:
summ_df["source"] = summ_df["dialogue"].apply(lambda x: "conversation-summarization: " + x)
summ_df["target"] = summ_df["summary"]
summ_df = summ_df[["source", "target"]]
summ_df

,source,target
0,"conversation-summarization: Hannah: Hey, do yo...",Hannah needs Betty's number but Amanda doesn't...
1,conversation-summarization: Eric: MACHINE!\r\n...,Eric and Rob are going to watch a stand-up on ...
2,"conversation-summarization: Lenny: Babe, can y...",Lenny can't decide which trousers to buy. Bob ...
3,"conversation-summarization: Will: hey babe, wh...",Emma will be home soon and she will let Will k...
4,"conversation-summarization: Ollie: Hi , are yo...",Jane is in Warsaw. Ollie and Jane has a party....
...,...,...
495,"conversation-summarization: Amber: Hi Erin, gu...",John and Annie are moving to London because he...
496,conversation-summarization: Vicky: Is everyone...,"They are meeting at Fratellis, upstairs."
497,conversation-summarization: Kaylin: <file_gif>...,Kaylin has set her alarm for tomorrow.
498,conversation-summarization: Keira: How come it...,Keira wonders why wildangel plays in the wrong...


In [23]:
data = load_dataset("paws", "labeled_final", split="test")
nli_df = data.to_pandas()
nli_df = nli_df.sample(500)

nli_df

,id,sentence1,sentence2,label
2810,2811,"After playing for Millwall , Clapton Orient an...","After playing in Italy for Millwall , Clapton ...",0
7292,7293,The route is now Pennsylvania Route 287 from T...,The route is now Pennsylvania Route 287 from L...,0
7459,7460,"43.0 % were German , 10.8 % Norwegian , 8.3 % ...","43.0 % were German , 10.8 % Norwegian , 8.3 % ...",0
4975,4976,Anchorage is 17 miles by air and 47 miles by r...,Wasilla is 17 miles by air and 47 miles on the...,0
2514,2515,The Buda River is a tributary of the Râmnicul ...,The river Râmnicul Sărat is a tributary of the...,0
...,...,...,...,...
1711,1712,The annual rainfall variation in the district ...,The annual rainfall variation in the district ...,1
4010,4011,"In January 2011 , the deputy general secretary...","In January 2011 , FIBA Asia deputy secretary g...",1
7679,7680,Warren Zevon wrote the song `` The Envoy '' fr...,"Habib wrote the song `` The Envoy '' , from hi...",0
2316,2317,"The former is tied by Hamazaki , the latter wa...",The former is tied by Hamazaki and the latter ...,1


In [24]:
nli_df["source"] = nli_df[["sentence1", "sentence2"]].apply(lambda x: "nli-hypothesis-check: sentence1: " + x["sentence1"] + " sentence2: " + x["sentence2"], axis=1)
nli_df

,id,sentence1,sentence2,label,source
2810,2811,"After playing for Millwall , Clapton Orient an...","After playing in Italy for Millwall , Clapton ...",0,nli-hypothesis-check: sentence1: After playing...
7292,7293,The route is now Pennsylvania Route 287 from T...,The route is now Pennsylvania Route 287 from L...,0,nli-hypothesis-check: sentence1: The route is ...
7459,7460,"43.0 % were German , 10.8 % Norwegian , 8.3 % ...","43.0 % were German , 10.8 % Norwegian , 8.3 % ...",0,nli-hypothesis-check: sentence1: 43.0 % were G...
4975,4976,Anchorage is 17 miles by air and 47 miles by r...,Wasilla is 17 miles by air and 47 miles on the...,0,nli-hypothesis-check: sentence1: Anchorage is ...
2514,2515,The Buda River is a tributary of the Râmnicul ...,The river Râmnicul Sărat is a tributary of the...,0,nli-hypothesis-check: sentence1: The Buda Rive...
...,...,...,...,...,...
1711,1712,The annual rainfall variation in the district ...,The annual rainfall variation in the district ...,1,nli-hypothesis-check: sentence1: The annual ra...
4010,4011,"In January 2011 , the deputy general secretary...","In January 2011 , FIBA Asia deputy secretary g...",1,nli-hypothesis-check: sentence1: In January 20...
7679,7680,Warren Zevon wrote the song `` The Envoy '' fr...,"Habib wrote the song `` The Envoy '' , from hi...",0,nli-hypothesis-check: sentence1: Warren Zevon ...
2316,2317,"The former is tied by Hamazaki , the latter wa...",The former is tied by Hamazaki and the latter ...,1,nli-hypothesis-check: sentence1: The former is...


In [25]:

hypo_dict = {
    0: "contradiction",
    1: "entailment"
}

nli_df["target"] = nli_df["label"].apply(lambda x: hypo_dict[x])
nli_df = nli_df[["source", "target"]]
nli_df

,source,target
2810,nli-hypothesis-check: sentence1: After playing...,contradiction
7292,nli-hypothesis-check: sentence1: The route is ...,contradiction
7459,nli-hypothesis-check: sentence1: 43.0 % were G...,contradiction
4975,nli-hypothesis-check: sentence1: Anchorage is ...,contradiction
2514,nli-hypothesis-check: sentence1: The Buda Rive...,contradiction
...,...,...
1711,nli-hypothesis-check: sentence1: The annual ra...,entailment
4010,nli-hypothesis-check: sentence1: In January 20...,entailment
7679,nli-hypothesis-check: sentence1: Warren Zevon ...,contradiction
2316,nli-hypothesis-check: sentence1: The former is...,entailment


In [26]:
list_of_dfs = [text_classification_df, qgen_df, qa_df, nli_df, summ_df, trans_df]

final_df = pd.concat(list_of_dfs, axis=0)
final_df

,source,target
0,text-classification-emotion: im feeling rather...,sadness
1,text-classification-emotion: im updating my bl...,sadness
2,text-classification-emotion: i never make her ...,sadness
3,text-classification-emotion: i left with my bo...,joy
4,text-classification-emotion: i was feeling a l...,sadness
...,...,...
258,"paraphrase: From 1863 to 1866 , Dunedin and Su...",Dunedin and Suburbs North was a parliamentary ...
2549,"paraphrase: The new year , the spring , the wi...","Also , `` The new year , the spring , the wine..."
6178,paraphrase: Sepahua is a district in southern ...,Sepahua is a district in southern Peru in the ...
5291,paraphrase: Karen Lieve Ria Hostens is married...,"Ria Hostens is married to J. Peter Burgess , c..."


In [27]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [28]:
train_df, test_df = train_test_split(final_df, test_size=0.1, random_state=0, shuffle=True)
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [29]:
model_id="google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [30]:
def preprocess_function(sample,padding="max_length"):
    model_inputs = tokenizer(sample["source"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["target"], max_length=128, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [31]:

train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
test_tokenized_dataset = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

Map:   0%|          | 0/4018 [00:00<?, ? examples/s]

Map:   0%|          | 0/447 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [32]:

lora_config = LoraConfig(
 r=8,
 lora_alpha=16,
 lora_dropout=0.1,
 bias="none",
 task_type="SEQ_2_SEQ_LM",
 target_modules=["q", "v"]
)

In [33]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [34]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 785,509,376 || trainable%: 0.3004


In [35]:
 model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
         

In [36]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [37]:
from huggingface_hub import notebook_login
notebook_login()

In [38]:
output_dir="flant5-large-aio"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=6,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)

In [39]:
model.config.use_cache = False

In [40]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset
)

In [41]:
trainer.train()

Step,Training Loss
670,1.120200


TrainOutput(global_step=670, training_loss=1.1202210383628732, metrics={'train_runtime': 535.1657, 'train_samples_per_second': 7.508, 'train_steps_per_second': 1.252, 'total_flos': 4644839557693440.0, 'train_loss': 1.1202210383628732, 'epoch': 1.0})

Trying to See the effect of the Fine tuned model over the original one.

In [42]:
peft_save_model_id="flant5-large-aio"
trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

In [56]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

input_text = "Is the sentiment of this sentence , happy or sad?: i don t feel guilty like i m not going to be able to cook for him"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


<pad> negative</s>


In [59]:
# pip install accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("Niharrrrrr/flant5-large-aio")
model = T5ForConditionalGeneration.from_pretrained("Niharrrrrr/flant5-large-aio", device_map="auto")

input_text = "Is the sentiment of this sentence happy or sad?: i don t feel guilty like i m not going to be able to cook for him"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


<pad> sad</s>
